In [1]:
import numpy as np
import os
from scipy.stats import norm
import math
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
from scipy.interpolate import interp1d
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve, root
import datetime
import matplotlib.ticker as mtick 
import datetime
import statsmodels.api as sm

In [ ]:
# latex_width = 6.69423 inches
w = 6.69423 * 9/10
golden_ratio = (5**.5 - 1) / 2
h = w * golden_ratio
h

In [10]:
maturities = np.array([0, 1,2,3,5,7,10,12,15,20,25])
swap_rates = np.array([0.042, 0.042, 0.043, 0.047, 0.054, 0.057, 0.06, 0.061, 0.059, 0.056, 0.0555])

maturities_2 = np.array([0,	0.99,1,1.99,2,2.99,3,3.99,5,6.99,7,9.99,10,11.99,12,14.99,15,19.99,20,24.99])
forw_flat = np.array([0.042,0.042,0.04400096,0.04400096,0.05504608,0.05504608,0.064587848,0.064587848,0.064537411,0.064537411,0.067033154,0.067033154,0.066014169,0.066014169,0.051037629,0.051037629,0.047050895,0.047050895,0.053502366,0.053502366])

delta = 0.01

def piecewise_linear(T, swap_rates):
    S = interp1d(maturities, 
                 swap_rates, 
                 kind='linear', 
                 fill_value='extrapolate')
    return S(T)

def disc_factor(T, swap_rates):
    return ((1 + piecewise_linear(T, swap_rates)) ** (-T))

def forw_rate_piecewise_linear(T, swap_rates):
    return ((disc_factor(T, swap_rates) / disc_factor(T+delta, swap_rates)) ** (1 / delta) - 1)

ts = pd.DataFrame(np.arange(0,25 + delta, delta))
curve = ts.copy()
forw_1 = ts.copy()

for i in range(len(ts)):
    T = ts.iloc[i]
    curve.iloc[i] = piecewise_linear(T, swap_rates)
    forw_1.iloc[i] = forw_rate_piecewise_linear(T,swap_rates)

samlet = pd.concat([ts, curve], axis = 1)
samlet_2 = pd.concat([ts,forw_1], axis = 1)

fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_xlim(0, 25)
ax.set_ylim(0.03,0.08)
ax.set_yticks(np.arange(0.03, 0.09, 0.01))
ax.grid(ls = (0,(1,2)))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1, decimals = 0))
plt.plot(samlet.iloc[:,0], samlet.iloc[:,1], label = 'Piecewise Linear Yield Curve', color ='r')
plt.plot(maturities_2, forw_flat, label = 'Piecewise Flat Forward Curve', color='g')
plt.plot(samlet_2.iloc[:,0], samlet_2.iloc[:,1], label = 'Piecewise Linear Forward Curve', color='b')
plt.xlabel('T (Years)')
plt.legend(loc = 'upper right',frameon=False,fontsize=11)
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/bootstrapped_curves.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_5272\3518476706.py:48: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [11]:
# cubic spline
maturities = np.array([0, 1,2,3,5,7,10,12,15,20,25])
swap_rates = np.array([0.042, 0.042, 0.043, 0.047, 0.054, 0.057, 0.06, 0.061, 0.059, 0.056, 0.0555])

delta = 0.01

def cubspline_curve(T, swap_rates):
    S = interp1d(maturities, 
                 swap_rates, 
                 kind='cubic', 
                 fill_value='extrapolate')
    return S(T)

def disc_factor(T, swap_rates):
    return (np.exp(- cubspline_curve(T, swap_rates) * T))

def forw_rate(T, swap_rates):
    return (disc_factor(T, swap_rates) / disc_factor(T+delta, swap_rates) - 1) / delta

ts = pd.DataFrame(np.arange(0,25 + delta, delta))
curve = ts.copy()
forw = ts.copy()

for i in range(len(ts)):
    date = datetime.date(2023,1,4)
    T = ts.iloc[i]
    curve.iloc[i] = cubspline_curve(T, swap_rates)
    forw.iloc[i] = forw_rate(T, swap_rates)

samlet = pd.concat([ts, curve], axis = 1)
samlet_2 = pd.concat([ts,forw], axis = 1)

fig, ax = plt.subplots(1,1, figsize=(w,h))

ax.set_xlim(0, 25)
ax.set_ylim(0.03,0.08)
ax.set_yticks(np.arange(0.03, 0.09, 0.01))
ax.grid(ls = (0,(1,2)))
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1, decimals = 0))
plt.plot(samlet.iloc[:,0], samlet.iloc[:,1], label = 'Yield Curve', color = 'r')
plt.plot(samlet_2.iloc[:,0], samlet_2.iloc[:,1], label = 'Forward Curve', color = 'b')
plt.xlabel('T (Years)')
plt.legend(bbox_to_anchor=(1.005, 0.96),frameon=False,fontsize=11)
fig.tight_layout()
plt.savefig('C:/Users/Mikkel/OneDrive/CBS/CBS - FIN/THESIS/Main/python/billeder/cubic_spline_curves.pgf')
plt.show()

C:\Users\Mikkel\AppData\Local\Temp\ipykernel_5272\2192763422.py:46: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
